In [ ]:
!pip install transformers
!pip install pandas
!pip install numpy
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
# Load the Marathi language dataset from a user-provided CSV file
file_path = input("Enter the path to the training CSV file: ")
df = pd.read_csv(file_path)

Enter the path to the training CSV file: /content/Bert2.0.csv


In [ ]:
df.head(10)

,sentence,Index
0,"लोकशाही तोलण्यासाठी संवाद महत्वाचा आहे, या संव...",1
1,देश म्हणजे केवळ चार दिशांच्या सीमांमधला जमिनीच...,1
2,माहितीच्या अधिकाराच्या आधारे एक समाज खरंच विका...,1
3,मागच्या आठवड्यात जमिनीवरचा न्याय्य हक्क नाकारण...,1
4,सामान्य माणसाला कट्टरवादी भूमिका घ्यायला सांगण...,1
5,मंडेला हे शांतीच्या महत्तम राजदूतांपैकी एक होत...,1
6,तुरुंगातून सुटका झाल्यावर मंडेला यांनी राष्ट्र...,1
7,"संसदेचे कार्य प्रचंड गुंतागुंतीचे, वैविध्यपूर्...",1
8,कायदेनिर्मिती प्रक्रियेमध्ये तसेच इतर बाबींची ...,1
9,संसदेच्या कायदेनिर्मिती व इतर कामकाजविषयक बाबी...,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1496 entries, 0 to 1495
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  1496 non-null   object
 1   Index     1496 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 23.5+ KB


In [ ]:
df.shape

(1496, 2)

In [ ]:
df.tail(10)

,sentence,Index
1486,तगलेल्या तीन नेत्यांपैकी ममता बॅनर्जी आणि जगनम...,1
1487,शरद पवार यांच्या नेतृत्वाखाली राष्ट्रवादीला २५...,1
1488,काँग्रेसबरोबर आघाडीतच राष्ट्रवादीला सत्ता मिळाली.,1
1489,एकत्र काँग्रेस पक्ष असताना राज्यात २०० पेक्षा ...,1
1490,२०१४ आणि २०१९ मध्ये भाजपने अनुक्रमे १२२ आणि १०...,1
1491,या तुलनेत राष्ट्रवादीचे सर्वाधिक ७१ आमदार निवड...,1
1492,ते ही काँग्रेसबरोबर आघाडीत निवडून आले होते.,1
1493,स्वबळावर लढताना १९९९ मध्ये ५८ तर २०१४ मध्ये ४१...,1
1494,क्रिकेटचा महाकुंभ म्हटला जाणारा विश्वचषक २०२३ ...,2
1495,विश्वचषकाची काउंटडाऊन सुरू झाले आहे.,2


In [ ]:
# Split the dataset into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['sentence'].values, df['Index'].values, test_size=0.2, random_state=42
)

In [ ]:
# Load the BERT tokenizer for Marathi language
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
# Tokenize the text data
train_encodings = tokenizer.batch_encode_plus(
    train_texts.tolist(),
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors='pt'
)
test_encodings = tokenizer.batch_encode_plus(
    test_texts.tolist(),
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors='pt'
)

In [ ]:
# Convert labels to integers
train_labels = train_labels.astype(int)
test_labels = test_labels.astype(int)

In [ ]:
# Create PyTorch datasets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))

In [ ]:
# Define the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=3)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual

In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
# Set device to GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
# Set up GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
# Set training parameters
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 3

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Training loop
model.train()
for epoch in range(num_epochs):
    train_loss = 0.0
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} - Training loss: {train_loss/len(train_loader)}")

# Evaluation on test set
model.eval()
predictions = []
true_labels = []
for batch in test_loader:
    input_ids, attention_mask, labels = batch
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    preds = torch.argmax(logits, dim=1).cpu().detach().numpy()
    predictions.extend(preds)
    true_labels.extend(labels.cpu().numpy())

# Calculate accuracy, loss, and other metrics
accuracy = accuracy_score(true_labels, predictions)
# Calculate the F1 score
f1 = f1_score(true_labels, predictions, average='weighted')
classification_report = classification_report(true_labels, predictions)
confusion_mat = confusion_matrix(true_labels, predictions)

# Print the results
print(f"Accuracy: {accuracy}")
print(f'F1 Score: {f1}')
print("Classification Report:")
print(classification_report)
print("Confusion Matrix:")
print(confusion_mat)

Epoch 1 - Training loss: 0.5720899850130081
Epoch 2 - Training loss: 0.19762919877966245
Epoch 3 - Training loss: 0.11198635059098402
Accuracy: 0.9733333333333334
F1 Score: 0.9732453734354802
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        96
           1       0.97      0.95      0.96       110
           2       0.97      0.97      0.97        94

    accuracy                           0.97       300
   macro avg       0.97      0.97      0.97       300
weighted avg       0.97      0.97      0.97       300

Confusion Matrix:
[[ 96   0   0]
 [  2 105   3]
 [  0   3  91]]
